In [1]:
from arcpy import *

In [7]:
env.workspace = r'D:\Data_arcpy\Data_ProgSIG_Ctr20\Data_ProgSIG_Ctr20'
env.overwriteOutput=True

Qst 1

In [25]:
for couche in ListFeatureClasses():
    print Describe(couche).spatialReference.name

Nord_Maroc_Degree
Nord_Maroc_Degree
Nord_Maroc_Degree
Nord_Maroc_Degree
Nord_Maroc_Degree


In [9]:
fc = "Communes_04.shp"
AddField_management(fc,"area","Double")     
expression1 = "{}".format("!SHAPE.area@SQUAREKILOMETERS!")    
CalculateField_management(fc, "area", expression1, "PYTHON" )

<Result 'D:\\Data_arcpy\\Data_ProgSIG_Ctr20\\Data_ProgSIG_Ctr20\\Communes_04.shp'>

In [24]:
'''def lisfeature():
    return [feature for feature in ListFeatureClasses()]

i=1
for couche in ListFeatureClasses():
    if 'c' in Describe(couche).name.upper():
        i+=1
        
print(i)'''

Cur = da.SearchCursor(fc, ["area", "pop04"])

row = Cur.next()
print row[0]


2.33297284295


QST 2

In [6]:
for champ in ListFields(fc):
    print "name:" + champ.name + ", type: "+ champ.type

name:FID, type: OID
name:Shape, type: Geometry
name:CODPR, type: Double
name:CODEC, type: String
name:LIBELLE, type: String
name:POP04, type: Double
name:ANALPH, type: Double
name:ACTV, type: Double
name:POV, type: Double
name:TYPE, type: String
name:Eleve_ing, type: String
name:area, type: Double


QST 3

In [9]:
with da.SearchCursor(fc,["LIBELLE"],
                     "LIBELLE LIKE '___A%'") as cursor:
    for row in cursor:
        print('{}'.format(row[0]))

CR AIT MALEK
CR AIT MIMOUNE
CR AIT OURIBEL
CR AIT SIBERNE
CR AIT YADINE
CR AIT ICHOU
CR AIT IKKOU
CR AIN SBIT
CR AIN JOHRA
CR AIT BELKACEM
CR AIT BOUYAHYA  EL HAJ
MU AIN EL AOUDA
CR AIN ATTIG
AR AGDAL RIYAD


In [10]:
with da.SearchCursor(fc,["TYPE","LIBELLE", "POP04"],
                     "TYPE='RURAL' and POP04> 10000") as cursor:
    for row in cursor:
        print('{} est {}, et sa pop : {}'.format(row[1],row[0], row[2]))

CR SIDI ABDERRAZAK est RURAL, et sa pop : 13654.0
CR AIT MIMOUNE est RURAL, et sa pop : 10236.0
CR AIT OURIBEL est RURAL, et sa pop : 10224.0
CR AIT YADINE est RURAL, et sa pop : 19461.0
CR ELGANZRA est RURAL, et sa pop : 13404.0
CR SIDI EL GHANDOUR est RURAL, et sa pop : 18587.0
CR AIT IKKOU est RURAL, et sa pop : 10676.0
CR AIN SBIT est RURAL, et sa pop : 11411.0
CR BRACHOUA est RURAL, et sa pop : 12371.0
CR EZZHILIGA est RURAL, et sa pop : 15506.0
CR LAGHOUALEM est RURAL, et sa pop : 12560.0
CR MARCHOUCH est RURAL, et sa pop : 11075.0
CR AIN JOHRA est RURAL, et sa pop : 10151.0
CR M QAM TOLBA est RURAL, et sa pop : 14705.0
CR SHOUL est RURAL, et sa pop : 19706.0
CR SIDI BOUKNADEL est RURAL, et sa pop : 34279.0
CR OUMAZZA est RURAL, et sa pop : 10530.0
CR SIDI YAHYA ZAER est RURAL, et sa pop : 28773.0
CR AIN ATTIG est RURAL, et sa pop : 17688.0
CR MERS EL KHEIR est RURAL, et sa pop : 14488.0
CR SABBAH est RURAL, et sa pop : 12912.0
CR MAJMAA TOLBA est RURAL, et sa pop : 16698.0


QST 5

In [11]:
with da.SearchCursor(fc,["LIBELLE"],
                     "Eleve_ing='ZIDI SALMA'") as cursor:
    for row in cursor:
        print('name: {} '.format(row[0]))

name: CR ELGANZRA 


QST 6

In [12]:
Dissolve_management(fc, "Provinces", "CODPR")   

<Result 'D:\\Data_arcpy\\Data_ProgSIG_Ctr20\\Data_ProgSIG_Ctr20\\Provinces.shp'>

In [13]:
def listingfeatures():
    return [Describe(feature).baseName for feature in ListFeatureClasses()]

print listingfeatures()

[u'Communes_04', u'Douars', u'Provinces']


qst 7

In [15]:
with da.SearchCursor(fc,["POP04", "AREA", "CODPR"],
                     "CODPR=291.0") as cursor:
    pop_tot=0
    super_tot=0
    for row in cursor:
        code = row[2]
        pop_tot+=row[0]
        super_tot+=row[1]
    print "population totale: {}, superficie: {} km² et le code du province est {}".format(pop_tot, super_tot, code) 

population totale: 521815.0, superficie: 7735.65179101 km² et le code du province est 291.0


QST 8

In [23]:
with da.SearchCursor(fc,[ "POP04", "AREA"],
                     "CODPR=291.0") as cursor:
    pop_tot=0
    super_tot = 0
    for row in cursor:
        pop_tot+=row[0]
        super_tot+=row[1]
    densite = pop_tot/super_tot
    print str(densite) + " habitants au Km²" 

AddField_management("Provinces.shp","densitePop","Double")        
#CalculateField_management("Provinces.shp", "densitePop", densite, "PYTHON" 
#with da.InsertCursor("Provinces.shp", ["CODPR", "densitePop"]) as cursor:
'''
for row in cursor:
        if row[0]==291.0:
            cursor.InsertCursor(densite)
'''
with da.UpdateCursor("Provinces.shp", ["CODPR", "densitePop"]) as cursor:
    for row in cursor:
        if row[0]==291.0:
            row[1]=densite
        else:
            row[1]=0
        cursor.updateRow(row)

67.4558542832 habitants au Km²


In [12]:
for elem in ListFields("Provinces.shp"):
    print elem.name

FID
Shape
CODPR
densitePop


QST9  

In [26]:
MakeFeatureLayer_management("Communes_04.shp","commune")
maCommune=SelectLayerByAttribute_management("commune" ,"NEW_SELECTION","Eleve_ing='ZIDI SALMA'" )
CopyFeatures_management(maCommune,"D:\Data_arcpy\Data_ProgSIG_Ctr20\Data_ProgSIG_Ctr20\Ma_commune")
Intersect_analysis (["douars.shp","Ma_commune.shp"], "Douars_Macommune.shp", "", 1.5, "point")

<Result 'D:\\Data_arcpy\\Data_ProgSIG_Ctr20\\Data_ProgSIG_Ctr20\\Douars_Macommune.shp'>

In [28]:
for champ in ListFields("Douars_Macommune.shp"):
    print champ.name

FID
Shape
FID_Douars
CODE_D
FID_Ma_com
CODPR
CODEC
LIBELLE
POP04
ANALPH
ACTV
POV
TYPE
Eleve_ing
area


QST10

In [3]:
with da.SearchCursor("Douars_Macommune.shp", ["FID"]) as cursor:
    nombre_douars = []
    for row in cursor:
        nombre_douars.append(row[0])
    print "nombre des douars est : {}".format(len(nombre_douars))

nombre des douars est : 29


QST 11

In [42]:
#on ajoute d'abord le point centroide dans une couche
with da.SearchCursor("Douars_Macommune.shp", "SHAPE@XY") as cursor:
    centroid_coords = []
    for row in cursor:
        centroid_coords.append(row[0])
    point = Point()
    pointGeometryList = []
    for pt in centroid_coords:
        point.X = pt[0]
        point.Y = pt[1]

    pointGeometry = PointGeometry(point)
    pointGeometryList.append(pointGeometry)
arcpy.CopyFeatures_management(pointGeometryList, "C:\Users\LAKTEB\Documents\ArcGIS\Default.gdb\point_centroid")

<Result 'C:\\Users\\LAKTEB\\Documents\\ArcGIS\\Default.gdb\\point_centroid'>

In [50]:
pointGeometry

<PointGeometry object at 0x5c736d0[0x5c746e0]>

In [52]:
#point centroide
with da.SearchCursor("C:\Users\LAKTEB\Documents\ArcGIS\Default.gdb\point_centroid", "SHAPE@XY") as cursor:
    centroid_coords = []
    for row in cursor:
        centroid_coords.append(row[0])
    print centroid_coords

[(449000.0001220703, 388000.0001220703)]


QST 12

In [65]:
Near_analysis("Douars_Macommune.shp", "C:\Users\LAKTEB\Documents\ArcGIS\Default.gdb\point_centroid")

<Result 'D:\\Data_arcpy\\Data_ProgSIG_Ctr20\\Data_ProgSIG_Ctr20\\Douars_Macommune.shp'>

In [67]:
for field in ListFields("Douars_Macommune.shp"):
    print field.name

FID
Shape
FID_Douars
CODE_D
FID_Ma_com
CODPR
CODEC
LIBELLE
POP04
ANALPH
ACTV
POV
TYPE
Eleve_ing
area
NEAR_FID
NEAR_DIST


In [103]:
with da.SearchCursor("Douars_Macommune.shp", ["NEAR_DIST"]) as cursor:
    near_dis = []
    for row in cursor:
        if row[0] != 4.13274392486e-09:
            near_dis.append(row[0])
    print "distance:", min(near_dis), "id:", near_dis.index(min(near_dis))

distance: 1414.21356237 id: 26
